<a href="https://colab.research.google.com/github/kentaojapi/trial-gemma-2-jpn/blob/main/trial_gemma_2_jpn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gemma-2-JPNを試してみる

https://huggingface.co/google/gemma-2-2b-jpn-it

In [2]:
pip install -U transformers torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 86.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 89.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2


In [3]:
import torch
from transformers import pipeline


pipe = pipeline(
    "text-generation",
    model="google/gemma-2-2b-jpn-it",
    model_kwargs={"torch_dtype": torch.bfloat16},
    device="cuda",  # replace with "mps" to run on a Mac device
)

config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/46.9k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

In [20]:
import time


def measure_time(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        print(f"execution sec: {execution_time:.4f}sec")
        return result
    return wrapper


@measure_time
def example_function(messages: str) -> None:
    outputs = pipe(messages, return_full_text=False, max_new_tokens=256)
    assistant_response = outputs[0]["generated_text"].strip()
    print(assistant_response)

## JSON出力

In [7]:
messages = [
    {"role": "user", "content": """
# 命令書
randomな数字を以下の形式で出力してください。
{"random_int": 999}
# 出力:
    """},
]
example_function(messages)

The 'max_batch_size' argument of HybridCache is deprecated and will be removed in v4.46. Use the more precisely named 'batch_size' argument instead.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


```json
{"random_int": 782}
```
execution sec: 3.4945sec


## コンテキスト検索のための位置づけ情報付与

In [22]:
DOCUMENT_CONTEXT_PROMPT = """
<document>
担当者A: こんにちは、B先輩。先日の会議で話題に上がった新規サービスの損害賠償条項について、少し気になる点があるのですが、お時間ありますか？

担当者B: もちろんです、A君。どんな点が気になっているの？

担当者A: はい。現在の契約書案では、当社の過失による損害に対して、直接損害のみを賠償対象としています。しかし、これでは顧客に対して十分な保護を提供できないのではないかと懸念しています。

担当者B: なるほど。確かにその点は議論の余地がありますね。ただ、間接損害まで賠償対象に含めると、当社のリスクが大きく増大する可能性があります。そのバランスをどう取るかが難しいところですね。

担当者A: おっしゃる通りです。ただ、競合他社の多くが間接損害も一部カバーしているという調査結果があります。このままでは、市場での競争力が落ちる可能性もあるのではないでしょうか。

担当者B: それは興味深い指摘ですね。確かに市場動向は無視できません。ただ、他社の対応をそのまま真似るのではなく、当社のビジネスモデルや財務状況に合わせた判断が必要だと思います。

担当者A: その通りですね。では、compromise案として、一定の上限を設けた上で間接損害も賠償対象に含める、というのはどうでしょうか？例えば、直接損害と間接損害を合わせて、契約金額の2倍を上限とするなど。

担当者B: それは良いアイデアかもしれません。上限を設けることで、当社のリスクを一定程度に抑えつつ、顧客にも追加の保護を提供できますからね。ただ、その場合、「間接損害」の定義を明確にする必要がありそうです。

担当者A: その点は私も同感です。例えば、逸失利益や営業損失などを具体的に列挙し、それ以外は除外する、といった形でしょうか。

担当者B: そうですね。さらに、間接損害の立証責任を顧客側に負わせる条項も入れておくと良いかもしれません。これにより、根拠のない過大な賠償請求を防ぐことができます。

担当者A: なるほど、それは重要な指摘ですね。立証責任の問題は見落としていました。ありがとうございます。

担当者B: いえいえ、こういった議論ができるのは良いことです。では、これらの点を踏まえて、新しい条項案を作成してみましょう。その後、経営陣にも相談して、最終的な判断を仰ぐのはどうでしょうか。

担当者A: はい、そうしましょう。この方針で進めていけば、顧客保護と当社リスクのバランスが取れた、競争力のある契約になると思います。早速、条項案の作成に取り掛かります。

担当者B: 素晴らしい。私も協力しますので、一緒に良い案を作り上げていきましょう。他に気になる点があれば、遠慮なく相談してくださいね。

担当者A: ありがとうございます。心強いです。それでは、作業を進めてまいります。

</document>
"""

CHUNK_CONTEXT_PROMPT = """

文書全体の中で位置づけたいチャンクは以下の通りです。
<chunk>
このままでは、市場での競争力が落ちる可能性もあるのではないでしょうか
</chunk>

このチャンクの検索性を向上させるために、このチャンクを文書全体の中で位置づけるための文脈を示してください。
文書全体の内容を簡単に要約しつつ、文脈を簡潔に答えてください。それ以外は答えないでください。
"""

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": DOCUMENT_CONTEXT_PROMPT,
            },
            {
                "type": "text",
                "text": CHUNK_CONTEXT_PROMPT
            }
        ]
    }
]
example_function(messages)


このチャンクは、新規サービスの損害賠償条項について、競合他社の対応を参考にすべきかどうかを議論している中で、市場での競争力の低下を懸念していることを示しています。
execution sec: 5.5681sec


## 検索クエリ内から単語の抜き出し

In [37]:
PROMPT = """
# 命令書:
以下に与えられるクエリから、BM25による全文検索を行うために重要と思われる単語を抜き出してください。
システムに組み込むため、絶対にJSON文字列以外は出力しないでください。
加えて以下に従ってください。

1. 絶対にJSON以外は出力しないでください。
2. 重要な名詞、動詞、形容詞を抽出してください。
3. 固有名詞（人名、組織名、地名など）を優先的に抽出してください。
4. 一般的な助詞、助動詞、接続詞は除外してください。
5. 各キーワードの重要度を1-10のスケールでスコアリングしてください。

出力は以下の様なJSON形式でお願いします。
{{
    "keywords": [
        {{
          "keyword": ""キーワード1",
          "score": 7
        }},
        {{
          "keyword": ""キーワード2",
          "score": 3
        }},
        ...
    ]
}}

# クエリ
{query}

# JSON:
"""

query = "人材紹介サービス会社に対しての報酬割合と支払い時期について過去の対応事例を知りたい"

messages = [
    {"role": "user", "content": PROMPT.format(query=query)},
]
example_function(messages)

```json
{
    "keywords": [
        {
          "keyword": "人材紹介サービス会社",
          "score": 9
        },
        {
          "keyword": "報酬割合",
          "score": 8
        },
        {
          "keyword": "支払い時期",
          "score": 7
        }
    ]
}
``` 



**解説:**

* **人材紹介サービス会社:** クエリの中心となるキーワード。
* **報酬割合:**  報酬に関する重要な要素。
* **支払い時期:**  報酬に関する重要な要素。
execution sec: 10.4486sec
